In [1]:
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import brightway2 as bw

In [2]:
# to import forwast
import os
import zipfile
from bw2data.utils import download_file

In [3]:
from premise import *

## This notebook

The objective of this notebook is to demonstrate how to solve the "My First LCA" exercise from the ESA course using the Brightway and ActivityBrowser tools. Additionally, it provides supplementary information on these tools.

- Author (and contact):
    - Alvaro Hahn (alvaro.hahn-menacho@psi.ch)


### Useful material

1. General info on how to use Brightway:
   - https://github.com/Depart-de-Sentier/teaching-material/tree/main/beginners
   - https://docs.brightway.dev/en/latest/
3. General info on Activity-Browser:
   - https://github.com/LCA-ActivityBrowser/activity-browser
   - https://www.youtube.com/@activity-browser
4. Interested in prospective LCA 🧐:
   - https://github.com/polca/premise

### 1. Set project and create biosphere

In [4]:
bd.projects.set_current("myfirstLCA_2") # Creating/accessing the projecs
bi.bw2setup()

Biosphere database already present!!! No setup is needed


In [5]:
bd.databases

Databases dictionary with 5 object(s):
	biosphere3
	ecoinvent 3.9.1 cutoff
	ecoinvent_cutoff_3.9_remind_SSP2-Base_2050
	ecoinvent_cutoff_3.9_remind_SSP2-PkBudg500_2050
	esa_exercise

## 2. Import forwast database (and ecoinvent)

In [13]:
ei_path = r"C:\Users\hahnme_a\Ecoinvent\3.9\datasets"
ei_name = 'ecoinvent 3.9.1 cutoff'

In [15]:
#ecoinvent
if ei_name in bd.databases:
    print("Database has already been imported.")
else:
    # mind that the ecoinvent file must be unzipped; then: path to the datasets subfolder
    # the "r" makes sure that the path is read as a string - especially useful when you have spaces in your string
    ei39cut = bi.SingleOutputEcospold2Importer(ei_path, ei_name)
    ei39cut
    ei39cut.apply_strategies()
    ei39cut.statistics()
    ei39cut.write_database()

Extracting ecospold2 files:
0% [##############################] 100% | ETA: 00:00:00 | Item ID: fffede58-fed2-5
Total time elapsed: 00:03:03


Title: Extracting ecospold2 files:
  Started: 04/16/2024 14:52:26
  Finished: 04/16/2024 14:55:29
  Total time elapsed: 00:03:03
  CPU %: 91.70
  Memory %: 18.10
Extracted 21238 datasets in 186.01 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_un

Writing activities to SQLite3 database:


21238 datasets
674593 exchanges
0 unlinked exchanges
  


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:39


Title: Writing activities to SQLite3 database:
  Started: 04/16/2024 14:55:36
  Finished: 04/16/2024 14:56:16
  Total time elapsed: 00:00:39
  CPU %: 99.40
  Memory %: 18.89
Created database: ecoinvent 3.9.1 cutoff


In [7]:
bd.databases

Databases dictionary with 2 object(s):
	biosphere3
	ecoinvent 3.9.1 cutoff

## 3. Import Excel database

In [8]:
imp = bi.ExcelImporter(os.path.join("data/Data_Import_Pallas.xlsx"))
imp.apply_strategies()
imp.match_database(fields=('name', 'unit', 'location','reference product'))
imp.match_database("ecoinvent 3.9.1 cutoff", fields=('name', 'unit', 'location','reference product'))
imp.statistics()

Extracted 1 worksheets in 0.03 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 5.35 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
23 datasets
153 exchanges
26 unlinked exchanges
  Type biosphere: 4 unique unlinked exc

(23, 153, 26)

### ⚠️ Warning: It is key to have 0 unlinked exchanges. Otherwise, it will not be possible to solve the system.

#### Only once we have 0 unliked exchanges we write our database. What if we need to troubleshoot? We run:

In [9]:
imp.write_excel()

Wrote matching file to:
C:\Users\hahnme_a\AppData\Local\pylca\Brightway3\myfirstLCA_2.0ef126b032886206d5b99c0d9d380032\output\db-matching-pallas2020_paper.xlsx


In [18]:
imp.write_database()

Writing activities to SQLite3 database:
0% [########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 04/16/2024 15:09:25
  Finished: 04/16/2024 15:09:25
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 19.32
Created database: esa_exercise


In [19]:
bd.databases

Databases dictionary with 3 object(s):
	biosphere3
	ecoinvent 3.9.1 cutoff
	esa_exercise

## 4. Exploring the database

In [6]:
wbi = bd.Database("esa_exercise")
for act in wbi:
    print(act)

'diesel supply' (liter, CH, None)
'natural gas supply' (cubic meter, CH, None)
'power generation' (kilowatt hour, CH, None)
'EV production' (unit, CH, None)
'ICEV production' (unit, CH, None)
'driving the ICEV' (kilometer, CH, None)
'battery manufacturing' (unit, CH, None)
'driving the EV' (kilometer, CH, None)


In [7]:
for key in bd.methods:
   if 'climate change' in key:
       print(key)

('CML v4.8 2016', 'climate change', 'global warming potential (GWP100)')
('Ecological Scarcity 2021', 'climate change', 'global warming potential (GWP100)')
('EF v3.0', 'climate change', 'global warming potential (GWP100)')
('EF v3.1', 'climate change', 'global warming potential (GWP100)')
('IMPACT 2002+ (Endpoint)', 'climate change', 'climate change')
('IMPACT 2002+ (Endpoint)', 'climate change', 'total')
('IPCC 2013', 'climate change', 'global temperature change potential (GTP100)')
('IPCC 2013', 'climate change', 'global temperature change potential (GTP20)')
('IPCC 2013', 'climate change', 'global warming potential (GWP100)')
('IPCC 2013', 'climate change', 'global warming potential (GWP20)')
('IPCC 2021', 'climate change', 'global temperature change potential (GTP100)')
('IPCC 2021', 'climate change', 'global temperature change potential (GTP50)')
('IPCC 2021', 'climate change', 'global warming potential (GWP100)')
('IPCC 2021', 'climate change', 'global warming potential (GWP20)'

## 5. LCA

#### Goal and scope

In [8]:
# functional unit (driving 100km with each vehicle alternative)
ev_driving = [act for act in wbi if 'driving the EV' in act['name'] and 'CH' in act['location']][0]
functional_unit_EV = {ev_driving:100}
functional_unit_EV

{'driving the EV' (kilometer, CH, None): 100}

In [9]:
icev_driving = [act for act in wbi if 'driving the ICEV' in act['name'] and 'CH' in act['location']][0]
functional_unit_icev = {icev_driving:100}
functional_unit_icev

{'driving the ICEV' (kilometer, CH, None): 100}

In [10]:
# Method: let's select EF3.0
CC_method = [m for m in bd.methods if 'EF v3.0' in str(m) and  'climate change' in str(m) and 'global warming potential (GWP100)' in str(m)][0]
CC_method

('EF v3.0 no LT',
 'climate change no LT',
 'global warming potential (GWP100) no LT')

#### LCI + LCIA

In [11]:
lca_ev = bc.LCA(functional_unit_EV,CC_method)
lca_ev.lci()
lca_ev.lcia()
lca_ev.score

11.232424827019575

In [12]:
lca_icev = bc.LCA(functional_unit_icev,CC_method)
lca_icev.lci()
lca_icev.lcia()
lca_icev.score

24.750991772952755

#### Interpretation

In [13]:
print('- Driving 100km using an electric car yields an impact of : {:.2f} kgCO2eq'.format(lca_ev.score))
print('- Driving 100km using a diesel car yields an impact of : {:.2f} kgCO2eq'.format(lca_icev.score))
if lca_ev.score < lca_icev.score:
    print('- Choosing an electric vehicle is the better alternative for climate change mitigation, resulting in a {:.2f}% reduction in impact compared to an internal combustion engine vehicle.'
          .format((lca_icev.score - lca_ev.score) / lca_icev.score * 100))
else:
    print('- Choosing an internal combustion engine vehicle is the better alternative for climate change mitigation, resulting in a {:.2f}% reduction in impact compared to an electric vehicle.'
          .format((lca_ev.score - lca_icev.score) / lca_ev.score * 100))

- Driving 100km using an electric car yields an impact of : 11.23 kgCO2eq
- Driving 100km using a diesel car yields an impact of : 24.75 kgCO2eq
- Choosing an electric vehicle is the better alternative for climate change mitigation, resulting in a 54.62% reduction in impact compared to an internal combustion engine vehicle.


## 6. Prospective LCA

Here comes premise

Upon the database extraction, you can import some of your Brightway2-compatible inventories like so:

In [6]:
clear_cache()

Cache folder cleared!


In [7]:
ndb = NewDatabase(
            scenarios=[
                {"model":"remind", "pathway":"SSP2-Base", "year":2050},
                {"model":"remind", "pathway":"SSP2-PkBudg500", "year":2050},
            ],
            source_db="ecoinvent 3.9.1 cutoff", 
            source_version="3.9.1",
            key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
            additional_inventories= [ # <-- this is NEW
                {"filepath": r"C:\Users\hahnme_a\Project Environments\my_first_LCA\data\LCI_cars_premise.xlsx",
                 "ecoinvent version": "3.9"}, # <-- this is NEW
            ] # <-- this is NEW
                 )

premise v.(2, 0, 2)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21238/21238 [00:00<00:00, 25021.06it/s]


Adding exchange data to activities


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 674593/674593 [01:34<00:00, 7139.70it/s]


Filling out exchange data


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21238/21238 [00:08<00:00, 2454.69it/s]


Set missing location of datasets to global scope.
Set missing location of production exchanges to scope of dataset.
Correct missing location of technosphere exchanges.
Correct missing flow categories for biosphere exchanges
Remove empty exchanges.
Remove uncertainty data.
- Extracting inventories
Cannot find cached inventories. Will create them now for next time...
Importing default inventories...

Extracted 1 worksheets in 0.84 seconds
Migrating to 3.8 first
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Remove uncertainty data.
Extracted 1 worksheets in 0.12 seconds
Migrating to 3.8 first
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Remove unce

In [8]:
ndb.update('electricity')

Updating: electricity: 100%|███████████| 1/1 [05:36<00:00, 336.79s/it]


Done!



In [ ]:
ndb.write_db_to_brightway()

Write new database(s) to Brightway.
Running all checks...
Running all checks...
Database ecoinvent_cutoff_3.9_remind_SSP2-Base_2050 already exists: it will be overwritten.
Vacuuming database 


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:55


Title: Writing activities to SQLite3 database:
  Started: 04/24/2024 10:26:25
  Finished: 04/24/2024 10:28:20
  Total time elapsed: 00:01:55
  CPU %: 98.40
  Memory %: 34.35


## LCIA